# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [115]:
from sklearn.metrics.pairwise import polynomial_kernel as pk
from sklearn.metrics.pairwise import check_pairwise_arrays
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from sklearn.utils.extmath import safe_sparse_dot

iris = load_iris()
data_set = iris.data
labels = iris.target
objects_count = len(train_labels)

In [116]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        d=2
        X = data_set
        coef0 = 1
        
        gamma = 1.0 / X.shape[1]
        kernel = np.dot(X, X.T)
        kernel *= gamma
        kernel += coef0
        kernel **= d
        #pass # put your code here
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [117]:
def choose_set_for_label(data_set, label):
    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

    train_labels[train_labels!=label] = -1
    test_labels[test_labels!=label] = -1

    train_labels[train_labels==label] = 1
    test_labels[test_labels==label] = 1
    return train_data_set, test_data_set, train_labels, test_labels

In [118]:
def get_labels_count(data_set):
    return len(data_set)

In [119]:
#Use the code that we have implemented earlier:

In [120]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    objects_count = get_labels_count(train_data_set)
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [121]:
LABEL = 0

In [122]:
# modify this part 
train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, LABEL)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  1.1726e+02 -1.7992e+03  4e+03  2e-01  2e-15
 1:  7.7967e+01 -1.6973e+02  3e+02  5e-03  2e-15
 2:  1.0169e+01 -2.2323e+01  3e+01  4e-16  3e-15
 3: -4.9607e-01 -4.9938e+00  4e+00  3e-16  2e-15
 4: -1.4344e+00 -2.5218e+00  1e+00  2e-16  5e-16
 5: -1.6897e+00 -2.1674e+00  5e-01  7e-16  3e-16
 6: -1.8078e+00 -2.0073e+00  2e-01  4e-16  2e-16
 7: -1.8464e+00 -1.9763e+00  1e-01  2e-16  2e-16
 8: -1.8825e+00 -1.9020e+00  2e-02  2e-16  2e-16
 9: -1.8902e+00 -1.8906e+00  4e-04  2e-16  2e-16
10: -1.8904e+00 -1.8904e+00  5e-06  5e-16  2e-16
11: -1.8904e+00 -1.8904e+00  5e-08  6e-16  3e-16
Optimal solution found.
0.26666666666666666


In [125]:
# modify this part 
train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, LABEL)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='linear')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0: -3.3341e+00 -2.9597e+03  7e+03  3e-01  4e-14
 1:  1.6979e+00 -2.2614e+02  4e+02  1e-02  4e-14
 2:  1.3388e+00 -4.2016e+00  7e+00  1e-04  8e-15
 3:  2.4861e-01 -4.0053e-01  7e-01  8e-07  4e-15
 4:  1.6637e-02 -7.5931e-02  9e-02  2e-16  3e-15
 5: -9.2499e-03 -2.0822e-02  1e-02  2e-16  1e-15
 6: -1.1198e-02 -1.5948e-02  5e-03  2e-16  5e-16
 7: -1.2340e-02 -1.6353e-02  4e-03  2e-16  3e-16
 8: -1.4778e-02 -1.5378e-02  6e-04  2e-16  3e-16
 9: -1.4977e-02 -1.5004e-02  3e-05  2e-16  3e-16
10: -1.4987e-02 -1.4987e-02  7e-07  2e-16  3e-16
11: -1.4987e-02 -1.4987e-02  8e-09  2e-16  3e-16
Optimal solution found.
0.7333333333333333


In [126]:
# modify this part 
train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, LABEL)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='poly')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  4.9630e+00 -1.5176e+03  3e+03  2e-01  4e-14
 1:  2.5701e+00 -1.6725e+02  3e+02  2e-02  4e-14
 2:  4.6951e-01 -1.9250e+01  4e+01  2e-03  2e-14
 3:  2.5910e-01 -8.6857e-01  1e+00  3e-05  1e-14
 4:  4.8658e-02 -8.5798e-02  1e-01  2e-07  5e-15
 5:  1.8470e-03 -1.7098e-02  2e-02  2e-16  3e-15
 6: -3.1145e-03 -6.3231e-03  3e-03  2e-16  9e-16
 7: -4.2480e-03 -6.3191e-03  2e-03  2e-16  5e-16
 8: -5.4103e-03 -5.9788e-03  6e-04  2e-16  5e-16
 9: -5.6081e-03 -5.6193e-03  1e-05  2e-16  6e-16
10: -5.6123e-03 -5.6125e-03  2e-07  2e-16  6e-16
11: -5.6124e-03 -5.6124e-03  2e-09  2e-16  7e-16
Optimal solution found.
0.7333333333333333
